This notebook is part of the *orix* documentation https://orix.readthedocs.io. Links to the documentation won’t work from the notebook.

# Crystal reference frame

This notebook describes the alignment of crystal reference frames and symmetry
operations in `orix`.

In [ ]:
%matplotlib inline

from diffpy.structure import Atom, Lattice, Structure
import matplotlib.pyplot as plt
import numpy as np

from orix.crystal_map import Phase
from orix.quaternion import Rotation
from orix.vector import Miller


plt.rcParams.update({
    "figure.figsize": (10, 5),
    "font.size": 20,
    "axes.grid": True,
    "lines.markersize": 10,
    "lines.linewidth": 3,
})

## Alignment and the structure matrix

The direct Bravais lattice is characterized by basis vectors ($\mathbf{a},
\mathbf{b}, \mathbf{c}$) with unit cell edge lengths ($a$, $b$, $c$) and angles
($\alpha$, $\beta$, $\gamma$). The reciprocal lattice has basis vectors given by

$$
\mathbf{a^*} = \frac{\mathbf{b} \times \mathbf{c}}{\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})},\:\:
\mathbf{b^*} = \frac{\mathbf{c} \times \mathbf{a}}{\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})},\:\:
\mathbf{c^*} = \frac{\mathbf{a} \times \mathbf{b}}{\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})},
$$

with reciprocal lattice parameters ($a^*$, $b^*$, $c^*$) and angles ($\alpha^*$,
$\beta^*$, $\gamma^*$).

Using these two crystallographic lattices, we can define a standard Cartesian
(orthonormal) reference frame by the unit vectors ($\mathbf{e_1}, \mathbf{e_2},
\mathbf{e_3}$). In principle, the direct lattice reference frame can be oriented
arbitrarily in the Cartesian reference frame. In `orix` we have chosen

$$
\mathbf{e_1} ||\: \frac{\mathbf{a}}{a},\:\:
\mathbf{e_2} ||\: \mathbf{e_3} \times \mathbf{e_1},\:\:
\mathbf{e_3} ||\: \frac{\mathbf{c^*}}{c^*}.
$$

This alignment is used for example in
<cite data-cite="rowenhorst2015consistent">Rowenhorst et al. (2015)</cite> and
<cite data-cite="degraef2003introduction">De Graef (2003)</cite>, the latter
which was the basis for the *EMsoft* Fortran suite of programs. Another common
option is $\mathbf{e_1} || \mathbf{a^*}/a^*$, $\mathbf{e_2} ||
\mathbf{e_3} \times \mathbf{e_1}$, $\mathbf{e_3} || \mathbf{c}/c$, which is used
for example in
<cite data-cite="britton2016tutorial">Britton et al. (2016)</cite> and the
`diffpy.structure` Python package, which we'll come back to.

In calculations, it is useful to describe the transformation of the Cartesian
unit *row* vectors to the coordinates of the direct lattice vectors by the
structure *row* matrix $\mathbf{A}$ (also called the crystal *base*). Given the
chosen alignment of basis vectors with the Cartesian reference frame,
$\mathbf{A}$ is defined as

\begin{equation}
\mathbf{A} = 
\begin{pmatrix}
a & 0 & 0\\
b\cos\gamma & b\sin\gamma & 0\\
c\cos\beta & -c\frac{(\cos\beta\cos\gamma - \cos\alpha)}{\sin\gamma} & \frac{\mathrm{V}}{ab\sin\gamma}
\end{pmatrix},
\end{equation}

where $V$ is the volume of the unit cell.

In `orix`, we use the
[Lattice](https://www.diffpy.org/diffpy.structure/mod_lattice.html#diffpy.structure.lattice.Lattice)
class in `diffpy.structure` to keep track of these properties internally. Let's
create a trigonal crystal with lattice parameters $(a, b, c)$ = (1.7, 1.7, 1.4)
nm and ($\alpha, \beta, \gamma$) = $(90^{\circ}, 90^{\circ}, 120^{\circ})$

In [ ]:
lattice = Lattice(1.7, 1.7, 1.4, 90, 90, 120)
lattice

`diffpy.structure` stores the structure matrix in the `Lattice.base` property

In [ ]:
lattice.base

and we see that `diffpy.structure` does not use the `orix` alignment mentioned
above, since $\mathbf{e1} \nparallel \mathbf{a} / a$. Instead, we see that
$\mathbf{e3} \parallel \mathbf{c} / c$, which is in line with the alternative
alignment mentioned above.

Thus, the alignment is updated internally whenever a
[Phase](reference.rst#orix.crystal_map.Phase) is created, a class which brings
together this crystal lattice and a point group
[Symmetry](reference.rst#orix.quaternion.Symmetry)

In [ ]:
phase = Phase(point_group="321", structure=Structure(lattice=lattice))
phase.structure.lattice.base

<div class="alert alert-warning">

Warning

Using another alignment than the one described above has undefined behaviour in
orix. Therefore, it is important that the structure matrix of a `Phase` instance
is not changed.

</div>

<div class="alert alert-info">

Note
    
The lattice is included in a
[Structure](https://www.diffpy.org/diffpy.structure/package.html#diffpy.structure.structure.Structure)
because the latter class brings together a lattice and
[atoms](https://www.diffpy.org/diffpy.structure/mod_atom.html#diffpy.structure.atom.Atom),
which is useful when simulating diffraction.

</div>

We can visualize the alignment of the direct and reciprocal lattice basis
vectors with the Cartesian reference frame using the stereographic projection

In [ ]:
abc = Miller(uvw=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], phase=phase)
abcr = Miller(hkl=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], phase=phase)

In [ ]:
fig = abc.scatter(c=["r", "g", "b"], marker="o", return_figure=True, axes_labels=["e1", "e2"], hemisphere="both")
abcr.scatter(c=["r", "g", "b"], marker="x", s=300, figure=fig)

## Alignment of symmetry operations

To see where the crystallographic axes about which the point group symmetry
operations rotate, we can add symmetry operations to the figure, like is done
in the [Visualizing point groups](point_groups.ipynb) user guide for all point
groups supported in `orix`

In [ ]:
ori = Rotation.from_axes_angles((0, 1, 0), np.deg2rad(65))
phase.point_group.plot(figure=fig, orientation=ori, fc="none", ec="C0", s=150)
fig

## Converting crystal vectors

The reference frame of the stereographic projection above is the Cartesian
reference frame ($\mathbf{e_1}, \mathbf{e_2}, \mathbf{e_3}$). The Cartesian
coordinates $(\mathbf{x}, \mathbf{y}, \mathbf{z})$ of
$(\mathbf{a}, \mathbf{b}, \mathbf{c})$ and
$(\mathbf{a^*}, \mathbf{b^*}, \mathbf{c^*})$ were found using $\mathbf{A}$ in
the following conversions

\begin{align}
(x, y, z) &= [uvw] \cdot \mathbf{A},\\
(x, y, z) &= (hkl) \cdot (\mathbf{A}^{-1})^T.
\end{align}

Let's compute the internal conversions directly and check for equality

In [ ]:
np.allclose(
    abc.data,  # (x, y, z)
    np.dot(abc.uvw, phase.structure.lattice.base)
)

In [ ]:
np.allclose(
    abcr.data,  # (x, y, z)
    np.dot(abcr.hkl, np.linalg.inv(phase.structure.lattice.base).T)
)